<a href="https://colab.research.google.com/github/ashjig/My-files2/blob/main/gptstocknifty50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# Function to load Nifty50 stock data
def load_nifty50_data():
    # Replace 'path/to/nifty50_data.csv' with the actual path to your dataset
    nifty50_data = pd.read_csv('path/to/nifty50_data.csv', parse_dates=True, index_col='Date')
    return nifty50_data

# Function to load constituent stocks data
def load_constituent_stocks_data():
    # Replace 'path/to/constituent_stocks_data.csv' with the actual path to your dataset
    stocks_data = pd.read_csv('path/to/constituent_stocks_data.csv', parse_dates=True, index_col='Date')
    return stocks_data

# Function to preprocess data
def preprocess_data(nifty50_data, stocks_data):
    # Combine Nifty50 and constituent stocks data
    combined_data = pd.concat([nifty50_data['Close'], stocks_data], axis=1)

    # Forward-fill missing values
    combined_data = combined_data.ffill()

    # Create weekly close data
    weekly_close = combined_data.resample('W-Fri').last()

    return weekly_close

# Function to create sequences for time series data
def create_sequences(data, seq_length):
    sequences, labels = [], []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data.iloc[i + seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# Load data
nifty50_data = load_nifty50_data()
constituent_stocks_data = load_constituent_stocks_data()

# Preprocess data
weekly_close_data = preprocess_data(nifty50_data, constituent_stocks_data)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(weekly_close_data)

# Set the sequence length
sequence_length = 10

# Create sequences and labels
X, y = create_sequences(pd.DataFrame(scaled_data), sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Set: {loss}')

# Make predictions on the test set
predictions = model.predict(X_test)

# Inverse transform the predictions and actual values to their original scale
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(predictions, label='Predicted Weekly Close')
plt.plot(y_test, label='Actual Weekly Close')
plt.legend()
plt.show()